In [248]:
#for analysis
import numpy as np
import pandas as pd


#for visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


#for machine learning algorithm
from sklearn.linear_model  import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [249]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
id = test_df['id']
train_df.head()

,id,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,9389_150,Y_1,Y,136,150,offline,intermediate,9389,M,Matriculation,3,24.0,5,N,1.0,0
1,16523_44,T_1,T,131,44,offline,easy,16523,F,High School Diploma,4,26.0,2,N,3.0,1
2,13987_178,Z_2,Z,120,178,online,easy,13987,M,Matriculation,1,40.0,1,N,2.0,1
3,13158_32,T_2,T,117,32,offline,easy,13158,F,Matriculation,3,NaN,4,N,1.0,1
4,10591_84,V_3,V,131,84,offline,intermediate,10591,F,High School Diploma,1,42.0,2,N,4.0,1


# Features categories

- categorical : program_id, program_type, test_type, difficulty_level,                 gender, education, city_tier,  total_program_enrolled,                 is_handicaped, trainee_engagement_rating, is_pass
- continuous :  age
- Discrete   :  program_duration, id, test_id, trainee_id,

In [250]:
print(train_df.columns.values)

['id' 'program_id' 'program_type' 'program_duration' 'test_id' 'test_type'
 'difficulty_level' 'trainee_id' 'gender' 'education' 'city_tier' 'age'
 'total_programs_enrolled' 'is_handicapped' 'trainee_engagement_rating'
 'is_pass']


In [251]:
train_df.shape

(73147, 16)

In [252]:
train_df.info()
print('*+'*20)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73147 entries, 0 to 73146
Data columns (total 16 columns):
id                           73147 non-null object
program_id                   73147 non-null object
program_type                 73147 non-null object
program_duration             73147 non-null int64
test_id                      73147 non-null int64
test_type                    73147 non-null object
difficulty_level             73147 non-null object
trainee_id                   73147 non-null int64
gender                       73147 non-null object
education                    73147 non-null object
city_tier                    73147 non-null int64
age                          45418 non-null float64
total_programs_enrolled      73147 non-null int64
is_handicapped               73147 non-null object
trainee_engagement_rating    73070 non-null float64
is_pass                      73147 non-null int64
dtypes: float64(2), int64(6), object(8)
memory usage: 8.9+ MB
*+*+*+*+*+*+*+*+*

age and trainee_engagement_rating features have null entries in both data frame

In [253]:
train_df.describe()

,program_duration,test_id,trainee_id,city_tier,age,total_programs_enrolled,trainee_engagement_rating,is_pass
count,73147.000000,73147.000000,73147.000000,73147.000000,45418.000000,73147.000000,73070.000000,73147.000000
mean,128.208676,91.079306,9843.079620,2.246845,36.494033,2.583934,2.395634,0.695408
std,6.889679,51.239048,5703.556093,1.011587,9.041119,1.240816,1.325288,0.460238
min,117.000000,0.000000,1.000000,1.000000,17.000000,1.000000,1.000000,0.000000
25%,121.000000,45.000000,5040.000000,1.000000,28.000000,2.000000,1.000000,0.000000
50%,131.000000,90.000000,9641.000000,2.000000,39.000000,2.000000,2.000000,1.000000
75%,134.000000,135.000000,14552.000000,3.000000,45.000000,3.000000,4.000000,1.000000
max,136.000000,187.000000,20098.000000,4.000000,63.000000,14.000000,5.000000,1.000000


In [254]:
train_df.describe(include=['O'])

,id,program_id,program_type,test_type,difficulty_level,gender,education,is_handicapped
count,73147,73147,73147,73147,73147,73147,73147,73147
unique,73147,22,7,2,4,2,5,2
top,11953_31,Y_1,Y,offline,easy,M,High School Diploma,N
freq,1,6687,22560,43543,40101,39043,34371,66577


In [255]:
train_df[['education','is_pass']].groupby(['education'],as_index=False).mean().sort_values(by='is_pass',ascending=False)

,education,is_pass
2,Masters,0.833984
0,Bachelors,0.742509
1,High School Diploma,0.714789
3,Matriculation,0.648592
4,No Qualification,0.583658


In [256]:
train_df[['gender','is_pass']].groupby(['gender'],as_index=False).mean().sort_values(by='is_pass',ascending=False)

,gender,is_pass
1,M,0.704172
0,F,0.685374


In [257]:
train_df[['difficulty_level','is_pass']].groupby(['difficulty_level'],as_index=False).mean().sort_values(by='is_pass',ascending=False)

,difficulty_level,is_pass
0,easy,0.737189
1,hard,0.667573
2,intermediate,0.654877
3,vary hard,0.430562


In [258]:
train_df[['test_type','is_pass']].groupby(['test_type'],as_index=False).mean().sort_values(by='is_pass',ascending=False)

,test_type,is_pass
1,online,0.779996
0,offline,0.637898


In [259]:
train_df[['trainee_engagement_rating','is_pass']].groupby(['trainee_engagement_rating'],as_index=False).mean().sort_values(by='is_pass',ascending=False)

,trainee_engagement_rating,is_pass
4,5.0,0.862422
3,4.0,0.804393
2,3.0,0.737904
1,2.0,0.685157
0,1.0,0.591522


In [260]:
print("before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['id','program_id','gender','trainee_id'],axis=1)
test_df = test_df.drop(['id','program_id','gender','trainee_id'],axis=1)
combine = [train_df, test_df]

"after",  train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

before (73147, 16) (31349, 15) (73147, 12) (31349, 11)


('after', (73147, 12), (31349, 11), (73147, 12), (31349, 11))

In [261]:
pd.crosstab(train_df['program_type'],train_df['is_pass'])

is_pass,0,1
program_type,,
S,602,724
T,5561,12897
U,2931,4890
V,5394,7631
X,619,2709
Y,5322,17238
Z,1851,4778


In [262]:
program_mapping = { "S":0, "T":1, "U":2, "V":3, "X":4,"Y":5, "Z":6}
train_df['program_type'] = train_df['program_type'].map(program_mapping)
test_df['program_type'] = test_df['program_type'].map(program_mapping)

train_df.head()

,program_type,program_duration,test_id,test_type,difficulty_level,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,5,136,150,offline,intermediate,Matriculation,3,24.0,5,N,1.0,0
1,1,131,44,offline,easy,High School Diploma,4,26.0,2,N,3.0,1
2,6,120,178,online,easy,Matriculation,1,40.0,1,N,2.0,1
3,1,117,32,offline,easy,Matriculation,3,NaN,4,N,1.0,1
4,3,131,84,offline,intermediate,High School Diploma,1,42.0,2,N,4.0,1


In [263]:
train_df['test_type'] = train_df['test_type'].map({"online":0, "offline":1}).astype(int)
test_df['test_type'] = test_df['test_type'].map({"online":0, "offline":1}).astype(int)
train_df.head()

,program_type,program_duration,test_id,test_type,difficulty_level,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,5,136,150,1,intermediate,Matriculation,3,24.0,5,N,1.0,0
1,1,131,44,1,easy,High School Diploma,4,26.0,2,N,3.0,1
2,6,120,178,0,easy,Matriculation,1,40.0,1,N,2.0,1
3,1,117,32,1,easy,Matriculation,3,NaN,4,N,1.0,1
4,3,131,84,1,intermediate,High School Diploma,1,42.0,2,N,4.0,1


In [264]:
difficulty_mapping = { "very hard":0, "intermediate":1, "hard":2, "easy":3}
train_df['difficulty_level'] = train_df['difficulty_level'].map(difficulty_mapping)
test_df['difficulty_level'] = test_df['difficulty_level'].map(difficulty_mapping)

train_df.head()

,program_type,program_duration,test_id,test_type,difficulty_level,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,5,136,150,1,1.0,Matriculation,3,24.0,5,N,1.0,0
1,1,131,44,1,3.0,High School Diploma,4,26.0,2,N,3.0,1
2,6,120,178,0,3.0,Matriculation,1,40.0,1,N,2.0,1
3,1,117,32,1,3.0,Matriculation,3,NaN,4,N,1.0,1
4,3,131,84,1,1.0,High School Diploma,1,42.0,2,N,4.0,1


In [265]:
education_mapping = { "No Qualification":0, "Matriculation":1, "High School Diploma":2, "Bachelors":3, "Masters":4}
train_df['education'] = train_df['education'].map(education_mapping)
test_df['education'] = test_df['education'].map(education_mapping)

train_df.head()

,program_type,program_duration,test_id,test_type,difficulty_level,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,5,136,150,1,1.0,1,3,24.0,5,N,1.0,0
1,1,131,44,1,3.0,2,4,26.0,2,N,3.0,1
2,6,120,178,0,3.0,1,1,40.0,1,N,2.0,1
3,1,117,32,1,3.0,1,3,NaN,4,N,1.0,1
4,3,131,84,1,1.0,2,1,42.0,2,N,4.0,1


In [266]:
train_df['is_handicapped'] = train_df['is_handicapped'].map({"N":0, "Y":1}).astype(int)
test_df['is_handicapped'] = test_df['is_handicapped'].map({"N":0, "Y":1}).astype(int)
train_df.head()

,program_type,program_duration,test_id,test_type,difficulty_level,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,5,136,150,1,1.0,1,3,24.0,5,0,1.0,0
1,1,131,44,1,3.0,2,4,26.0,2,0,3.0,1
2,6,120,178,0,3.0,1,1,40.0,1,0,2.0,1
3,1,117,32,1,3.0,1,3,NaN,4,0,1.0,1
4,3,131,84,1,1.0,2,1,42.0,2,0,4.0,1


In [267]:
train_df['age'].value_counts(dropna=False).head()

NaN      27729
 45.0     4043
 44.0     3723
 28.0     3211
 43.0     3136
Name: age, dtype: int64

In [268]:
train_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='ffill', inplace=True)

In [269]:
train_df.info()
print("--"*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73147 entries, 0 to 73146
Data columns (total 12 columns):
program_type                 73147 non-null int64
program_duration             73147 non-null int64
test_id                      73147 non-null int64
test_type                    73147 non-null int64
difficulty_level             73147 non-null float64
education                    73147 non-null int64
city_tier                    73147 non-null int64
age                          73147 non-null float64
total_programs_enrolled      73147 non-null int64
is_handicapped               73147 non-null int64
trainee_engagement_rating    73147 non-null float64
is_pass                      73147 non-null int64
dtypes: float64(3), int64(9)
memory usage: 6.7 MB
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31349 entries, 0 to 31348
Data columns (total 11 columns):
program_type                 31349 non-null i

In [270]:
# train_df.fillna(method='bfill')
# train_df.info()

In [271]:
# train_df['age'].fillna(train_df['age'].mean())
# train_df.info()

In [272]:
# from sklearn.model_selection import train_test_split

# X = train_df.drop(['is_pass'], axis=1)
# Y = train_df['is_pass']
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [273]:
test_df.head(5)

,program_type,program_duration,test_id,test_type,difficulty_level,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating
0,1,131,45,1,1.0,1,3,46.0,2,0,4.0
1,5,135,130,0,3.0,3,3,46.0,4,0,4.0
2,5,120,146,0,3.0,1,3,46.0,2,0,3.0
3,3,122,72,1,3.0,2,1,46.0,2,0,2.0
4,3,122,71,1,1.0,1,3,46.0,2,0,2.0


In [274]:
X_train = train_df.drop("is_pass", axis=1)
Y_train = train_df["is_pass"]
X_test  = test_df.copy()
X_train.shape, Y_train.shape, X_test.shape

((73147, 11), (73147,), (31349, 11))

In [275]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

95.56

In [277]:
submission = pd.DataFrame({
    "id": id,
    "is_pass":Y_pred
})
submission.to_csv("submission.csv",index=False)